[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/umatter/EDFB/blob/main/notebooks/Python/EDFB_Digital_Finance_%26_Banking_Logistic_Regression.ipynb)

# EDFB - Digital Finance & Banking - Logistic Regression

---

This notebook demonstrates how to train and evaluate a logistic regression classifier in Python. It is prepared for GitHub + Colab sharing: the first code cell bootstraps exact dependencies, and the dataset is loaded from the repo (or downloaded) so it runs end-to-end without manual uploads.

Tested on: Colab Python 3.10, Ubuntu 22.04


In [ ]:
# @title Setup (installs exact versions; safe to rerun)
import sys
!pip -q install --upgrade pip
!pip -q install "numpy>=2.0.0,<2.1.0" pandas==2.2.2 "scikit-learn>=1.6.0" matplotlib==3.9.0 seaborn==0.13.2

# Imports
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io
from pathlib import Path
import urllib.request
%matplotlib inline

print('Python:', sys.version)
print('Loaded versions:')
print('- numpy', np.__version__)
print('- pandas', pd.__version__)
import sklearn, matplotlib
print('- scikit-learn', sklearn.__version__)
print('- matplotlib', matplotlib.__version__)
print('- seaborn', sns.__version__)


In [ ]:
# To make this notebook's output stable across runs (we make the output reproducable)
np.random.seed(42)

In [ ]:
# Load dataset 'banking.csv' from repo if available; otherwise download
data_path = Path("data/banking.csv")
data_path.parent.mkdir(parents=True, exist_ok=True)
if data_path.exists():
    print("Loading data from", data_path)
    dataset = pd.read_csv(data_path)
else:
    url = "https://raw.githubusercontent.com/umatter/EDFB/main/data/banking.csv"
    try:
        print("Attempting to download banking.csv from", url)
        urllib.request.urlretrieve(url, data_path)
        dataset = pd.read_csv(data_path)
        print("Downloaded to", data_path)
    except Exception as e:
        raise RuntimeError(f"Could not obtain banking.csv: {e}")

In [ ]:
# Data is loaded into `dataset` above.

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
# Define set of numerical and categorical variables
num_var = dataset.drop(columns=['y']).select_dtypes([np.number]).columns
cat_var = dataset.drop(columns=['y']).select_dtypes(include=object).columns

In [ ]:
num_var

In [ ]:
cat_var

In [ ]:
# Check NAs
dataset.isna().any()

In [ ]:
# Get basic statistics for numerical variables
dataset.describe()

In [ ]:
# Check dispersion with box plot
from sklearn import preprocessing
def box_plot(df, standardize=True):

    fig=plt.figure(figsize=(20,10))

    if standardize==True:
        # standardize columns for better visualization
        df=pd.DataFrame(preprocessing.StandardScaler().fit_transform(df.values), columns = df.columns)
    fig=sns.boxplot(x='value', y='variable', data=pd.melt(df.reset_index(), id_vars='index', value_vars=list(df.columns)),
               orient='h')
    fig.tick_params(labelsize=10)
    fig.set_xlabel('')
    fig.set_ylabel('')
    fig.set_title('Note that variables are standardized\nfor better visualization', fontsize=20)
    plt.show()


box_plot(dataset[num_var], standardize=True)

In [ ]:
# We remove duration, pdays and previous
dataset=dataset.drop(columns=['duration', 'pdays', 'age', 'campaign', 'previous'])
num_var= dataset.drop(columns=['y']).select_dtypes([np.number]).columns

In [ ]:
# Check distribution for target variable
plt.figure(figsize=(10,10))
sns.catplot(x='y', kind="count", data=dataset) # categorical plots
plt.show()

In [ ]:
# Dataset is very unbalanced so we remove some observation for y=0 to be equal to 2*size of y=1.
# This is called "undersampling"

# We keep all y=1
from sklearn.model_selection import train_test_split
data_1 = dataset[dataset['y'] == 1]
print(data_1.shape)

# We take y=0 as double the size of data_1
# Moreover we "stratify" the sampling in order to take the same distribution for each variable
# We use the train_test_split function and we keep the test only
all_data_0 = dataset[dataset['y'] == 0]
percentage_corresponding_to_double_size = 2*data_1.shape[0] / all_data_0.shape[0] # 2*size_1 compared to size_0

X = all_data_0.drop(columns=['y'])
y = all_data_0['y'].to_frame()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)
data_0_big, data_0_small = train_test_split(all_data_0, test_size=percentage_corresponding_to_double_size,
                                                    random_state=0, shuffle=True)
print(data_0_big.shape) # remaining from the dataset
print(data_0_small.shape)


In [ ]:
# Merge two dataset

dataset=pd.concat([data_1, data_0_small], axis= 0).reset_index(drop=True)  # axis = 1 by column and = 0 by row
print(dataset.shape)

In [ ]:
# Check distribution for target variable after downsampling

plt.figure(figsize=(10,10))
sns.catplot(x='y', kind="count", data=dataset)
plt.show()

In [ ]:
# Plot the distribution and the boxplot of the numerical variables included in the dataset compared to the target (it's only 0 or 1)

fig = plt.figure(figsize=(15,30))
plot_count=1

# scale variable for better visualizing boxplot
dataset_scaled=pd.DataFrame(preprocessing.StandardScaler().fit_transform(dataset[num_var].values),columns = num_var)
dataset_scaled['y']=dataset['y'].astype(str)
y_1 = dataset.loc[dataset['y'] == 1] #.loc - access group of values using labels.
y_0 = dataset.loc[dataset['y'] == 0]

for var in num_var:
    # plot variable distribution
    ax = fig.add_subplot(math.ceil(len(num_var) / 2), 2, plot_count)
    sns.distplot(y_1[var], label='1', ax=ax)
    sns.distplot(y_0[var], label='0', ax=ax)
    ax.set_title('Distribution of ' + var, fontsize=20)
    ax.tick_params(labelsize=15)
    ax.set_xlabel('')
    ax.legend(fontsize=16)
    plot_count += 1


In [ ]:
# Check the distribution of the levels of the categorical variables compared with the target

fig = plt.figure(figsize=(15,30))
plot_count=1

for var in cat_var:
    # plot variable distribution
    ax = fig.add_subplot(math.ceil(len(cat_var) / 2), 2, plot_count)
    plot_set = dataset.groupby([var, 'y']).size().reset_index().pivot(columns='y', index=var, values=0)
    plot_set=plot_set.div(plot_set.sum(axis=1), axis=0).plot(kind='barh', stacked=True, ax=ax)
    ax.set_title('Target variable distribution for each\nlevel (' + str(len(dataset[var].unique())) +
                 ') of ' + var, fontsize=20)
    ax.tick_params(labelsize=15)
    ax.set_ylabel('')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=16)
    plot_count += 1
plt.tight_layout()
plt.show()

In [ ]:
dataset.head()

In [ ]:
# Create dummy variables & standardize the dataset
dataset_dummy=pd.get_dummies(dataset.copy(), dummy_na=False, drop_first=True) # Whether to get k-1 dummies out of k categorical levels by removing the first level.
dataset_dummy[num_var]=pd.DataFrame(preprocessing.StandardScaler().fit_transform(dataset[num_var].values),columns = num_var)

In [ ]:
dataset_dummy.head()

In [ ]:
# Check the correlations between variables
corrmat = dataset_dummy.corr()


In [ ]:
# Correlation matrix in key-values pairs
corrmat *= np.where(np.tri(*corrmat.shape, k=-1)==0, np.nan, 1)  # puts NaN on upper triangular matrix, including diagonal (k=-1)
corrmat_list=corrmat.unstack().to_frame()

# Check highest correlations
corrmat_list.columns=['correlation']
corrmat_list['abs_corr']=corrmat_list.correlation.abs()
corrmat_list.sort_values(by=['abs_corr'], ascending=False, na_position='last', inplace=True)
corrmat_list.drop(columns=['abs_corr']).head(10)


In [ ]:
# Plot correlation heatmap
plt.figure(figsize=(20,20))
sns.heatmap(corrmat, cmap ="YlGnBu", linewidths = 0.1)
plt.show()

In [ ]:
# Drop highly correlated columns
dataset_original=dataset.copy() # save original dataset

# Rename dataset_dummy and drop columns
col_to_drop=['emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed', 'loan_unknown', 'housing_unknown']
dataset=dataset_dummy.drop(columns=col_to_drop)
num_var=dataset.columns.intersection(num_var)

In [ ]:
# Ready to train and test our clasifer!
X = dataset.drop(columns=['y'])
y = dataset['y'].values
print(X.shape)
print(y.shape)

In [ ]:
# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=dataset['y'])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=0) # solver (https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451)
model.fit(X_train, y_train) # training the algorithm

In [ ]:
# Get fitted value on test set
y_test_predicted = model.predict(X_test)

# Compare predictions
display(pd.DataFrame({'True': y_test.flatten(), 'Predicted': y_test_predicted.flatten()}))

# Compare predicted probabilities (default threshold for converting to 0 or 1 is 0.5)
y_test_predicted_prob = model.predict_proba(X_test)[:,1]
display(pd.DataFrame({'True': y_test.flatten(), 'Predicted_prob': y_test_predicted_prob.flatten(), 'Predicted': y_test_predicted.flatten()}))

In [ ]:
# Evaluate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_predicted)

In [ ]:
# Evaluate confusion matrix

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Only use the labels that appear in the data
    classes = ['0', '1']
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_test_predicted)
plt.show()

In [ ]:
# Evaluate precision, recall, F1-score on train set
# A macro-average will compute the metric independently for each class and then take the average (hence treating all classes equally),
# whereas a micro-average will aggregate the contributions of all classes to compute the average metric.
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_predicted))

In [ ]:
# Evaluate ROC curve

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y_test, y_test_predicted)
fpr, tpr, thresholds = roc_curve(y_test, y_test_predicted_prob)


plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()